<a href="https://colab.research.google.com/github/interritus141/COMP0031-Group-Research-Project/blob/master/COMP0031.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

## Installation

## General

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

## Data

In [ ]:
from ta import add_all_ta_features
import yfinance as yf

## Environment

In [ ]:
import gym 
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3 import A2C, DDPG, DQN, PPO

## Backtesting

In [ ]:
from backtesting import Backtest, Strategy

# Data

## Tech Indicators

In [ ]:
def add_ta(df):
  ta_df = add_all_ta_features(df, open="Open", high="High", low="Low", close="Close", volume="Volume")
  # print(ta_df.columns)
  ta_df = ta_df[["Open", "High", "Low", "Close", "Volume", "Adj Close", "volume_obv",
                                "volume_adi", "trend_adx", "momentum_ao", "trend_macd", "momentum_rsi", 
                                "momentum_stoch"]]
  ta_df = ta_df.fillna(ta_df.mean())
  return ta_df

## Stocks

1. Apple Inc. (AAPL)
2. Microsoft Corp. (MSFT)
3. Amazon.com, Inc. ( AMZN)
4. Tesla, Inc. (TSLA)
5. Nvidia Corp. (NVDA)

In [ ]:
stock_volatilities = {}
stocks = dict.fromkeys(["AAPL", "MSFT", "AMZN", "TSLA", "NVDA", "CAAS"])

for stock in stocks.keys():
  stock_df = yf.download(stock, start="2018-01-01", end="2022-12-31", keepna=True)
  stock_df = stock_df.fillna(stock_df.mean())
  stock_df = add_ta(stock_df)
  stocks[stock] = stock_df
  stock_volatilities[stock] = None



In [ ]:
def add_volatility(df, df_name):
  df["Log returns"] = np.log(df['Close'] / df['Close'].shift())
  stock_volatilities[df_name] = df['Log returns'].std() * 252 ** .5


In [ ]:
def visualise_volatility(df, df_name, volatility):
  fig, ax = plt.subplots()
  df['Log returns'].hist(ax=ax, bins=50, alpha=0.6, color='b')
  ax.set_xlabel("Log return")
  ax.set_ylabel("Freq of log return")
  ax.set_title("{:s} volatility: {:.2f}%".format(df_name, volatility*100))

for stock, stock_df in stocks.items():
  add_volatility(stock_df, stock)
  # visualise_volatility(stock_df, stock, stock_volatilities[stock]) // use to generate graphs

print(stock_volatilities)

In [ ]:
def add_cov(df):
  df = df.reset_index()

  cov_list = []
  return_list = []

  # look back is one year
  lookback=252
  for i in range(lookback,len(df.index.unique())):
    data_lookback = df.iloc[i-lookback:i,:]
    price_lookback=data_lookback.pivot_table(index = 'Date', values = 'Close')
    return_lookback = price_lookback.pct_change().dropna()
    return_list.append(return_lookback)

    covs = return_lookback.cov().values 
    cov_list.append(covs)


  df_cov = pd.DataFrame({'Date':df["Date"].unique()[lookback:],'cov_list':cov_list,'return_list':return_list})
  df = df.merge(df_cov, on='Date')
  df = df.sort_values(['Date']).reset_index(drop=True)
  return df

In [ ]:
aapl_df = add_cov(aapl_df)
caas_df = add_cov(caas_df)

In [ ]:
new_aapl_df = aapl_df.copy()
new_aapl_df["tic"] = "AAPL"
new_caas_df = caas_df.copy()
new_caas_df["tic"] = "CAAS"
mixed_df = pd.concat([new_aapl_df, new_caas_df])

# Environment

In [ ]:
class StockPortfolioEnv(gym.Env):
  def __init__(self):
    pass
  
  def reset(self):
    pass
  
  def step(self, action):
    pass

## Kaggle Tutorial

In [ ]:
class StockPortfolioEnv(gym.Env):
  def __init__(self, 
                df,
                stock_dim,
                hmax,
                initial_amount,
                transaction_cost_pct,
                reward_scaling,
                state_space,
                action_space,
                tech_indicator_list,
                turbulence_threshold=None,
                lookback=252,
                day = 0):
    #super(StockEnv, self).__init__()
    #money = 10 , scope = 1
    self.day = day
    self.lookback=lookback
    self.df = df
    self.stock_dim = stock_dim
    self.hmax = hmax
    self.initial_amount = initial_amount
    self.transaction_cost_pct =transaction_cost_pct
    self.reward_scaling = reward_scaling
    self.state_space = state_space
    self.action_space = action_space
    self.tech_indicator_list = tech_indicator_list

    # action_space normalization and shape is self.stock_dim
    self.action_space = gym.spaces.Box(low = -1, high = 1,shape = (self.action_space,)) 
    # Shape = ??
    # covariance matrix + technical indicators
    self.observation_space = gym.spaces.Box(low=0, high=np.inf, shape = (1+len(self.tech_indicator_list),self.state_space))

    # load data from a pandas dataframe
    self.data = self.df.loc[self.day,:]
    self.covs = [[x[0][0] for x in self.data['cov_list']]]
    self.state =  np.append(np.array(self.covs), [self.data[tech].values.tolist() for tech in self.tech_indicator_list ], axis=0)
    self.terminal = False     
    self.turbulence_threshold = turbulence_threshold        
    # initalize state: inital portfolio return + individual stock return + individual weights
    self.portfolio_value = self.initial_amount

    # memorize portfolio value each step
    self.asset_memory = [self.initial_amount]
    # memorize portfolio return each step
    self.portfolio_return_memory = [0]
    self.actions_memory=[[1/self.stock_dim]*self.stock_dim]
    self.date_memory=[self.data["Date"].unique()[0]]


        
  def step(self, actions):
    # print(self.day)
    self.terminal = self.day >= len(self.df.index.unique())/self.stock_dim-1
    # print(actions)

    if self.terminal:
      df = pd.DataFrame(self.portfolio_return_memory)
      df.columns = ['daily_return']
      plt.plot(df.daily_return.cumsum(),'r')
      # plt.savefig('results/cumulative_reward.png')
      plt.close()

      plt.plot(self.portfolio_return_memory,'r')
      # plt.savefig('results/rewards.png')
      plt.close()

      print("=================================")
      print("begin_total_asset:{}".format(self.asset_memory[0]))           
      print("end_total_asset:{}".format(self.portfolio_value))

      df_daily_return = pd.DataFrame(self.portfolio_return_memory)
      df_daily_return.columns = ['daily_return']
      if df_daily_return['daily_return'].std() !=0:
        sharpe = (252**0.5)*df_daily_return['daily_return'].mean()/ \
                  df_daily_return['daily_return'].std()
        print("Sharpe: ",sharpe)
      print("=================================")

      return self.state, self.reward, self.terminal,{}

    else:
      #print("Model actions: ",actions)
      # actions are the portfolio weight
      # normalize to sum of 1
      #if (np.array(actions) - np.array(actions).min()).sum() != 0:
      #  norm_actions = (np.array(actions) - np.array(actions).min()) / (np.array(actions) - np.array(actions).min()).sum()
      #else:
      #  norm_actions = actions
      weights = self.softmax_normalization(actions) 
      #print("Normalized actions: ", weights)
      self.actions_memory.append(weights)
      last_day_memory = self.data

      #load next state
      self.day += 1
      self.data = self.df.loc[self.day,:]
      self.covs = [[x[0][0] for x in self.data['cov_list']]]
      self.state =  np.append(np.array(self.covs), [self.data[tech].values.tolist() for tech in self.tech_indicator_list ], axis=0)
      #print(self.state)
      # calcualte portfolio return
      # individual stocks' return * weight
      portfolio_return = sum(((self.data["Close"].values / last_day_memory["Close"].values)-1)*weights)
      # update portfolio value
      new_portfolio_value = self.portfolio_value*(1+portfolio_return)
      self.portfolio_value = new_portfolio_value

      # save into memory
      self.portfolio_return_memory.append(portfolio_return)
      self.date_memory.append(self.data["Date"].unique()[0])            
      self.asset_memory.append(new_portfolio_value)

      # the reward is the new portfolio value or end portfolo value
      self.reward = new_portfolio_value 
      #print("Step reward: ", self.reward)
      #self.reward = self.reward*self.reward_scaling

    return self.state, self.reward, self.terminal, {}

  def reset(self):
    self.asset_memory = [self.initial_amount]
    self.day = 0
    self.data = self.df.loc[self.day,:]
    # load states
    self.covs = [[x[0][0] for x in self.data['cov_list']]]
    self.state =  np.append(np.array(self.covs), [self.data[tech].values.tolist() for tech in self.tech_indicator_list ], axis=0)
    self.portfolio_value = self.initial_amount
    #self.cost = 0
    #self.trades = 0
    self.terminal = False 
    self.portfolio_return_memory = [0]
    self.actions_memory=[[1/self.stock_dim]*self.stock_dim]
    self.date_memory=[self.data["Date"].unique()[0]] 
    return self.state
    
  def render(self, mode='human'):
    return self.state
      
  def softmax_normalization(self, actions):
    numerator = np.exp(actions)
    denominator = np.sum(np.exp(actions))
    softmax_output = numerator/denominator
    return softmax_output

  
  def save_asset_memory(self):
    date_list = self.date_memory
    portfolio_return = self.portfolio_return_memory
    #print(len(date_list))
    #print(len(asset_list))
    df_account_value = pd.DataFrame({'Date':date_list,'daily_return':portfolio_return})
    return df_account_value

  def save_action_memory(self):
    # date and close price length must match actions length
    date_list = self.date_memory
    df_date = pd.DataFrame(date_list)
    df_date.columns = ['Date']
    
    action_list = self.actions_memory
    df_actions = pd.DataFrame(action_list)
    df_actions.columns = self.data["tic"]
    df_actions.index = df_date["Date"]
    #df_actions = pd.DataFrame({'date':date_list,'actions':action_list})
    return df_actions

  def _seed(self, seed=None):
    self.np_random, seed = seeding.np_random(seed)
    return [seed]

  def get_sb_env(self):
    e = DummyVecEnv([lambda: self])
    obs = e.reset()
    return e, obs

In [ ]:
stock_dimension = len(mixed_df["tic"].unique())
state_space = 2

env_kwargs = {
  "hmax": 100, 
  "initial_amount": 1000000, 
  "transaction_cost_pct": 0.001, 
  "state_space": state_space, 
  "stock_dim": stock_dimension, 
  "tech_indicator_list": [
    "volume_obv",
    "volume_adi", 
    "trend_adx", 
    "momentum_ao", 
    "trend_macd", 
    "momentum_rsi", 
    "momentum_stoch"
  ], 
  "action_space": stock_dimension, 
  "reward_scaling": 1e-4
    
}

env = StockPortfolioEnv(df = mixed_df, **env_kwargs)

In [ ]:
env_train, _ = env.get_sb_env()
print(type(env_train))

# Train

In [ ]:
policy = "MlpPolicy"

In [ ]:
a2c_agent = A2C(policy, env)
ddpg_agent = DDPG(policy, env)
# dqn_agent = DQN(policy, env)
ppo_agent = PPO(policy, env)

In [ ]:
print("A2C")
a2c_result = a2c_agent.learn(
    total_timesteps=1000,
)

print("DDPG")
ddpg_result = ddpg_agent.learn(
    total_timesteps=1000,
)

print("PPO")
ppo_result = ppo_agent.learn(
    total_timesteps=1000,
)

# Predict

In [ ]:
a2c_vec_env = a2c_result.get_env()
obs = a2c_vec_env.reset()
for i in range(1000):
    action, _states = a2c_result.predict(obs, deterministic=True)
    obs, reward, done, info = a2c_vec_env.step(action)
    # print("Step", i, reward)
    a2c_vec_env.render()

# Backtest

## To explore

In [ ]:
class MLTrainOnceStrategy(Strategy):
    price_delta = .004  # 0.4%

    def init(self):        
        # agent
        self.agent = ppo_result
        self.vec_env = ppo_result.get_env()
        self.obs = self.vec_env.reset()
        
        # data
        # self.data = mixed_df
        # self.data.set_index("Date")

        # Prepare empty, all-NaN forecast indicator
        self.forecasts = self.I(lambda: np.repeat(np.nan, len(self.data)), name='forecast')
    
    def next(self):
        # Proceed only with out-of-sample data. Prepare some variables
        high, low, close = self.data.High, self.data.Low, self.data.Close
        # current_time = self.data.index[-1]

        # Forecast the next movement
        action, _states = self.agent.predict(self.obs, deterministic=True)
        self.obs, forecast, done, info = self.vec_env.step(action)
        self.vec_env.render()

        # Update the plotted "forecast" indicator
        self.forecasts[-1] = forecast

        # If our forecast is upwards and we don't already hold a long position
        # place a long order for 20% of available account equity. Vice versa for short.
        # Also set target take-profit and stop-loss prices to be one price_delta
        # away from the current closing price.
        upper, lower = close[-1] * (1 + np.r_[1, -1]*self.price_delta)

        if forecast > 1000000 and not self.position.is_long:
            self.buy(size=.2, tp=upper, sl=lower)
        elif forecast <= 1000000 and not self.position.is_short:
            self.sell(size=.2, tp=lower, sl=upper)

        # Additionally, set aggressive stop-loss on trades that have been open 
        # for more than two days
        # for trade in self.trades:
        #     if current_time - trade.entry_time > pd.Timedelta('2 days'):
        #         if trade.is_long:
        #             trade.sl = max(trade.sl, low)
        #         else:
        #             trade.sl = min(trade.sl, high)

In [ ]:
bt = Backtest(aapl_df, MLTrainOnceStrategy, commission=.0002, margin=.05)
bt.run()